<a href="https://colab.research.google.com/github/mmsamiei/just-practice-deep/blob/master/digi_contest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import numpy as np

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
class SentimentModel(nn.Module):
  
  def __init__(self, hid_size, vocab_size, n_head, n_layers, pf_size, max_len, device):
    super().__init__()

    self.device = device
    
    self.hid_size = hid_size
    self.pf_size = pf_size
    self.max_len = max_len

    self.embedding = nn.Embedding(vocab_size, hid_size)

    self.position_enc = nn.Embedding(self.max_len, self.hid_size)
    self.position_enc.weight.data = self.position_encoding_init(self.max_len, self.hid_size)
    self.scale = torch.sqrt(torch.FloatTensor([self.hid_size])).to(device)

    self.layer_norm = nn.LayerNorm(self.hid_size)
    self.encoder_layer = nn.TransformerEncoderLayer(d_model=hid_size, nhead = n_head, dim_feedforward = pf_size)
    self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=n_layers, norm=self.layer_norm)
    self.fc = nn.Linear(hid_size, 2)

    self._init_weights()
  
  def forward(self, x):
    sent_len, batch_size = x.shape[0], x.shape[1]

    temp = x
    temp = self.embedding(temp)

    pos = torch.arange(0,sent_len).unsqueeze(1).repeat(1,batch_size).to(self.device)
    temp_pos_emb = self.position_enc(pos)

    temp = temp * self.scale + temp_pos_emb
    temp = self.encoder(temp)
    temp = self.fc(temp[0,:])
    return temp

  def _init_weights(self):
    for p in self.parameters():
      if p.dim() > 1:
        nn.init.xavier_uniform_(p)

  def append_decoder_layer(self):
    appended_mod = nn.TransformerEncoderLayer(d_model=self.hid_size, nhead = self.n_head, dim_feedforward = self.pf_size).to(self.device)
    for p in appended_mod.parameters():
      if p.dim() > 1:
        nn.init.xavier_uniform_(p)
    self.encoder.layers.append(appended_mod)
    self.encoder.num_layers += 1

  
  def position_encoding_init(self, n_position, d_pos_vec):
    ''' Init the sinusoid position encoding table '''

    # keep dim 0 for padding token position encoding zero vector
    position_enc = np.array([
        [pos / np.power(10000, 2*i/d_pos_vec) for i in range(d_pos_vec)]
        if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])

    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2]) # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2]) # dim 2i+1
    temp = torch.from_numpy(position_enc).type(torch.FloatTensor)
    temp = temp.to(self.device)
    return temp

In [0]:
hid_size = 16
vocab_size = 60 
n_head = 4
n_layers = 2
pf_size = 64
max_len = 1200
model = SentimentModel(hid_size, vocab_size, n_head, n_layers, pf_size, max_len, device).to(device)

In [4]:
test_len = 20
batch_size = 64
test_input = torch.LongTensor(test_len, batch_size).random_(1,vocab_size).to(device)
model(test_input).shape

torch.Size([64, 2])

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 30,066 trainable parameters


**Data Loading**

In [0]:
import torchtext
from torchtext.data import Field, SubwordField
from torchtext.data import Pipeline
import spacy

tokenize = lambda x: x.split()

def remove_repetitive_char(x):
  string = " "
  for char in x:
    if(char != string[-1]):
      string += char
  return string[1:]

def replace_bad_char(x):
  string = x
  string = string.replace("ة", "ه")
  string = string.replace("ي", "ی")
  string = string.replace("ٱ", "ا")
  string = string.replace("ڋ", "د")
  string = string.replace("ڑ", "ر")
  string = string.replace("ڪ", "ک")
  string = string.replace("ھ", "ه")
  string = string.replace("ی", "ی")
  string = string.replace("ە", "ه")
  string = string.replace("ﭘ", "پ")
  string = string.replace("ﮊ", "ژ")
  string = string.replace("ﮓ", "گ")
  string = string.replace("ﮔ", "گ")
  string = string.replace("ﺎ", "گ")
  string = string.replace("ﺖ", "گ")
  string = string.replace("ﺤ", "گ")
  string = string.replace("ﺪ", "گ")
  string = string.replace('ﺮ',"ر" )
  string = string.replace('ﺯ',"ز" )
  string = string.replace('ﺰ',"ز" )
  string = string.replace('ﺳ',"س" )
  string = string.replace('ﺴ',"س" )
  string = string.replace('ﺶ', "ش")
  string = string.replace('ﺷ', "ش")
  string = string.replace('ﺸ', "ش")
  string = string.replace('ﺼ', "ص")
  string = string.replace('ﺽ', "ض")
  string = string.replace('ﻌ', "ع")
  string = string.replace('ﻔ', "ف")
  string = string.replace('ﻗ', "ق")
  string = string.replace('ﻛ', "ک")
  string = string.replace('ﻜ', "ک")
  string = string.replace('ﻝ', "ل")
  string = string.replace('ﻟ',"ل" )
  string = string.replace('ﻢ',"م" )
  string = string.replace('ﻣ', "م")
  string = string.replace('ﻥ', "ن")
  string = string.replace('ﻦ', "ن")
  string = string.replace('ﻧ', "ن")
  string = string.replace('ﻨ', "ن")
  string = string.replace('ﻩ', "ه")
  string = string.replace('ﻪ', "ه")
  string = string.replace('ﻫ', "ه")
  string = string.replace('ﻬ', "ه")
  string = string.replace('ﻭ', "و")
  string = string.replace('ﻮ', "و")
  string = string.replace('ﻱ', "ی")
  string = string.replace('ﻲ', "ی")
  string = string.replace('ﻳ', "ی")
  return string
  


pre_pipe = Pipeline(remove_repetitive_char)
pre_pipe.add_before(replace_bad_char)

TEXT = Field(sequential=True, tokenize=tokenize, lower=True, preprocessing=pre_pipe, init_token="<bos>")
LABEL = Field(sequential=False, use_vocab=False)
ID = Field(sequential=False, use_vocab=False)

SUBTEXT = SubwordField()

In [0]:
from torchtext.data import TabularDataset

tv_datafields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("title", None), ("comment", TEXT),
                 ("rate", None), ("verification_status", LABEL)]

tst_datafields = [("id", ID), # we won't be needing the id, so we pass in None as the field
                 ("title", None), ("comment", TEXT),
                 ("rate", None)]



In [0]:
import random

def my_filter_pred(example, limited_word = 100):
  if(len(example.comment) <= limited_word):
    return True
  else:
    return False


def my_filter_pred_rand(example, limited_word = 100):
  if(len(example.comment) <= limited_word):
    if(random.random()<0.4):
      return True
    else:
      return False
  else:
    return False


train_dataset = TabularDataset(
               path='train_comments.csv',
               format='csv',
               skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=tv_datafields,
               filter_pred = my_filter_pred)


test_dataset = TabularDataset(
               path='test_nolabel_comments.csv',
               format='csv',
               skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=tst_datafields)




In [9]:
len(train_dataset)

172540

In [0]:
s = set()
for i, batch in enumerate(train_dataset):
  s = s.union(set(" ".join(batch.comment)))

In [11]:
s

{' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 '¡',
 '«',
 '°',
 '·',
 '»',
 '×',
 '÷',
 '͢',
 'ε',
 'κ',
 'г',
 'ժ',
 'հ',
 'ձ',
 'օ',
 'ּ',
 'פ',
 '،',
 '؛',
 '\u061c',
 '؟',
 'ء',
 'آ',
 'أ',
 'ؤ',
 'إ',
 'ئ',
 'ا',
 'ب',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ـ',
 'ف',
 'ق',
 'ك',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ى',
 'ً',
 'ٌ',
 'ٍ',
 'َ',
 'ُ',
 'ِ',
 'ّ',
 'ْ',
 'ٓ',
 'ٔ',
 'ٕ',
 '٠',
 '١',
 '٢',
 '٣',
 '٤',
 '٥',
 '٦',
 '٧',
 '٨',
 '٩',
 '٪',
 '٫',
 '٬',
 'ٰ',
 'ٵ',
 'ٹ',
 'پ',
 'ڃ',
 'چ',
 'ڔ',
 'ڗ',
 'ژ',
 'ک',
 'گ',
 'ۀ',
 'ہ',
 'ۂ',
 'ی',
 'ے',
 '۔',
 '۰'

In [12]:
train_dataset[0].__dict__.keys()

dict_keys(['comment', 'verification_status'])

In [13]:
train_dataset[0].verification_status

'0'

In [0]:
TEXT.build_vocab(train_dataset, min_freq=3)

In [15]:
len(TEXT.vocab)

30038

In [0]:
from torchtext.data import BucketIterator, interleave_keys, Iterator

batch_size = 512

train_iterator = BucketIterator(dataset= train_dataset, batch_size=batch_size,
                                device=device,
                                sort_key=lambda x: len(x.comment),
                                sort = True,
                                shuffle = True,
                                repeat = False)

test_iter = Iterator(test_dataset, batch_size=1, device=device, sort=False, sort_within_batch=False, repeat=False)


In [17]:
for batch in iter(train_iterator):
  print(batch.comment.shape)

torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([3, 512])
torch.Size([3, 512])
torch.Size([3, 512])
torch.Size([3, 512])
torch.Size([3, 512])
torch.Size([3, 512])
torch.Size([3, 512])
torch.Size([3, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([4, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([5, 512])
torch.Size([6, 512])
torch.Size([6, 512])
torch.Size([6, 512])
torch.Size([6, 512])
torch.Size([6, 512])
torch.Size([6, 512])
torch.Size([6, 512])
torch.Size([6, 512])
torch.Size([6, 512])
torch.Size([6, 512])
torch.Size([6

**train section**

In [0]:
vocab_size = len(TEXT.vocab)
hid_size = 128
pf_size = 256
n_head = 8
n_layer= 3
model = SentimentModel(hid_size, vocab_size, n_head, n_layers, pf_size, max_len, device).to(device)

In [19]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,396,418 trainable parameters


In [0]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
    
    def zero_grad(self):
        self.optimizer.zero_grad()

In [0]:
from tqdm import tqdm

def train_one_epoch(model,train_iter, optimizer, criterion, clip):
  epoch_loss = 0
  model.train()
  for batch in tqdm(train_iter):
    optimizer.zero_grad()
    batch_text = batch.comment
    batch_target = batch.verification_status
    result = model(batch_text)
    loss = criterion(result, batch_target.view(-1))
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
  return epoch_loss / len(train_iter)


In [0]:
def train(model, train_iter, optimizer, criterion, clip, N_EPOCH):
  for epoch in range(N_EPOCH):
    epoch_loss = train_one_epoch(model, train_iter, optimizer, criterion, clip)
    print("epoch is {} loss is {}".format(epoch, epoch_loss))

In [23]:
optimizer = NoamOpt(hid_size, 1, 2000,
              torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
criterion = torch.nn.CrossEntropyLoss()
train(model, train_iterator, optimizer, criterion, 1, 50)

  0%|          | 1/337 [00:00<00:39,  8.59it/s]

epoch is 0 loss is 0.33566401872330676


 53%|█████▎    | 180/337 [00:07<00:09, 17.02it/s]


KeyboardInterrupt: ignored

In [0]:
model.eval()
test_preds = []
with torch.no_grad():
  for batch in iter(test_iter):
    row_id = batch.id.item()
    if(batch.comment.shape[0]==0):
      res_out = 0
      test_preds.append([row_id,res_out])
      continue
    res = model(batch.comment)
    res_out = torch.argmax(res).item()
    test_preds.append([row_id,res_out])

In [0]:
test_preds.sort(key=lambda x: x[0])

In [0]:
import pandas
df = pandas.DataFrame(test_preds, columns= ['id', 'verification_status'])
df.to_csv("./file.csv", sep=',',index=False)

In [0]:
model.append_decoder_layer()